# Instalación del programa en python

In [1]:
# Install dependencies
%pip install -q amplpy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 31.6 MB/s eta 0:00:00


### Google colab & Kaggle integration

In [2]:
from amplpy import AMPL, ampl_notebook, Environment

ampl = ampl_notebook(
    modules=["highs", "cbc", "gurobi"],  # modules to install
    license_uuid="default",  # license to use
)  # instantiate AMPL object and register magics

# Taller

## Punto 1

### Minas de aluminio


Mejorar

In [29]:
%%ampl_eval
reset; # limpiar definiciones anteriores
# definir el conjunto de Lugares
set L;

# definir el conjunto de Fábricas
set F;

# definir el tipo de aluminio
set A;

#Definir el subconjunto de lugares
set LF{F} within L;

# definir el costo de la Bautista para los lugares
param costBau{LF[1]};

# definir la capacidad anual de la de operación de la bautista en los lugares
param capBauOper{LF[1]};

# definir los costos fijos de la operacion de las minas de las bautista en los lugares
param costFijosBau{LF[1]};

# definir capcidad anual de procesamiento la bautista en los lugares
param capProceAlum{LF[2]};

# definir los costos fijos de la operacion de las fabricas de aluminia en los lugares
param costFijosAlum{LF[2]};

# definir los costos de produccion en los lugares del tipo de alumina
param costProdAlum{LF[2],A};

# definir el rendimiento de aluminia en el lugar del tipo aluminia
param rendAlum{LF[1],A};

# definir el rendimiento del esmaltado en el lugar del tipo aluminia
param rendEsmal{LF[3],A};

# definir los costos de produccion en el lugar del tipo aluminia
param costProdEsmal{LF[3],A};

# definir la capacidad anual de procesamiento en el lugar del tipo aluminia
param capProceEsmal{LF[3]};

# definir los costos fijos de operación de las fabricas de esmaltado en los lugares
param costFijosEsmal{LF[3]};

# definir la demanda minima de aluminio esmaltado en los lugaros
param demandaMin{LF[3]};

# definir la demanda maxima minima de aluminio esmaltado en los lugaros
param demandaMax{LF[3]};

# definir la venta de aluminio esmaltado en los lugares
param ventaEsmal{LF[3]};

# definir los costos de tranporte de la Bautista de los lugares
param costTransBau{LF[1],LF[2]};

#definir los costos de tranporte de Aluminia de los lugares
param costTransAlumi{LF[2],LF[3]};

#definir los costos de tranporte del aluminio esmaltado de los lugares
param costTransEsmal{LF[3],LF[3]};

## M
param M;

# definir las variables de decision:
#cantidad de Bautista procesada en el lugar
var xb{LF[1],LF[2]} >= 0;

#cantidad de Aluminia procesada en el lugar
var xa{LF[2], LF[3],A} >= 0;

#cantidad de aluminio esmaltado procesada en el lugar
var xe{LF[3],LF[3],A} >= 0;

#apertura  de las fabriacas de Bautista en el lugar
var yb{LF[1]} >= 0;

#apertura de las fabriacas de Aluminia  en el lugar
var ya{LF[2]} >= 0;

#apertura de las fabriacas de Esmaltado en el lugar
var ye{LF[3]} >= 0;

# La función objetivo es maximizar la ganancia
maximize maxganancia: sum{l1 in LF[3], l2 in LF[3], a in A} xe[l1,l2,a]*(ventaEsmal[l2]-costProdEsmal[l1,a]-costTransEsmal[l1,l2])-sum{l1 in LF[2],l2 in LF[2], a in A} xa[l1,l2,a]*(costProdAlum[l1,a]+costTransAlumi[l1,l2])-sum{l1 in LF[1], l2 in LF[2]} xb[l1,l2]*(costBau[l1]+costTransBau[l1,l2])- sum{l in LF[1]} costFijosBau[l]*yb[l]-sum{l in LF[2]} costFijosAlum[l]*ya[l]-sum{l in LF[3]} costFijosEsmal[l]*ye[l];


# restricciones:

subject to capacidad_maxima_bauxita_minas {l1 in LF[1]}: sum{l2 in LF[2]} xb[l1,l2]  <= capBauOper[l1];

subject to capacidad_bauxita_procesamineto {l2 in LF[2]}: sum{l1 in LF[1]} xb[l1,l2]  <= capProceAlum[l2];

subject to capacidad_alumina_esmaltado {l2 in LF[3]}: sum{l1 in LF[2], a in A} xa[l1,l2,a]  <= capProceEsmal[l2];

subject to demanda_minima_esmaltado {l2 in LF[3]}: sum{l1 in LF[3], a in A} xe[l1,l2,a] >= demandaMin[l2];

subject to demanda_maxima_esmaltado {l2 in LF[3]}: sum{l1 in LF[3], a in A} xe[l1,l2,a] <= demandaMax[l2];

subject to igualda_rendimiento_bauxita_alum {l2 in LF[2]}: sum{l1 in LF[1], a in A} xb[l1,l2]*rendAlum[l1,a] = sum{l1 in LF[3], a in A} xa[l2,l1,a];

subject to igualda_rendimiento_alum_esmaltado {l2 in LF[3]}: sum{l1 in LF[1], a in A} xa[l1,l2,a]*rendEsmal[l2,a] = sum{l1 in LF[3], a in A} xe[l2,l1,a];

subject to apertura_minas_bauxita {l in LF[1]}: sum{l2 in LF[2]} xb[l,l2] <= M*yb[l];

subject to apertura_fabrica_alumina {l in LF[2]}: sum{l2 in LF[3], a in A} xa[l,l2,a] <= M*ya[l];

subject to apertura_fabrica_esmaltado {l in LF[3]}: sum{l2 in LF[3], a in A} xe[l,l2,a] <= M*ye[l];

In [30]:
import pandas as pd

ampl.set["L"] = ["A", "B", "C", "D", "E"]
ampl.set["F"] = [i+1 for i in range(3)]
ampl.set["A"] = [i+1 for i in range(3)]
# Cargar el DataFrame en AMPL como un conjunto indexado

subset_values = {1:["A", "B", "C"], 2:["B", "C", "D", "E"], 3:["D", "E"]}

In [31]:
%%ampl_eval
data;

set LF[1] := "A" "B" "C";
set LF[2] := "B" "C" "D" "E";
set LF[3] := "D" "E";



In [32]:
# Convertir subset_values a una lista de listas


ampl.param["costBau"]        = {"A":420, "B":360, "C":540}
ampl.param["capBauOper"]     = {"A":36000, "B":52000, "C":28000}
ampl.param["costFijosBau"]   = {"A":1500000, "B":2000000, "C":1250000}

ampl.param["capProceAlum"]   = {"B":40000, "C":20000, "D":30000, "E":80000}
ampl.param["costFijosAlum"]  = {"B":3000000, "C":2500000, "D":4800000, "E":6000000}

ampl.param["costProdAlum"]   = pd.DataFrame({1:[330, 320, 380, 240],
                                             2:[300, 290, 310, 215],
                                             3:[380, 320, 435, 300]},
                                            ["B", "C", "D", "E"])

ampl.param["rendAlum"]       = pd.DataFrame({1:[0.06, 0.08, 0.062],
                                             2:[0.065, 0.087, 0.07],
                                             3:[0.045, 0.06, 0.05]},
                                            ["A", "B", "C"])

ampl.param["rendEsmal"]      = pd.DataFrame({1:[0.4 for i in range(2)],
                                             2:[0.28 for i in range(2)],
                                             3:[0.5 for i in range(2)]},
                                            ["D", "E"])

ampl.param["costProdEsmal"]  = pd.DataFrame({1:[8500,5200],
                                             2:[6000,3700],
                                             3:[10000,6350]},
                                            ["D", "E"])

ampl.param["capProceEsmal"]  = {"D":4000, "E":7000}
ampl.param["costFijosEsmal"] = {"D":6500000, "E":8000000}
ampl.param["demandaMin"]     = {"D":1000, "E":1000}
ampl.param["demandaMax"]     = {"D":1300, "E":1500}
ampl.param["ventaEsmal"]     = {"D":60000, "E":57000}

ampl.param["costTransBau"]   = pd.DataFrame({"B":[400, 10, 1630],
                                             "C":[2010, 630, 10],
                                             "D":[510, 220, 620],
                                             "E":[1920, 1510, 940]},
                                            ["A", "B", "C"])

ampl.param["costTransAlumi"] = pd.DataFrame({"D":[220, 620, 0, 1465],
                                             "E":[1510, 940, 1615, 0]},
                                            ["B", "C", "D", "E"])

ampl.param["costTransEsmal"] = pd.DataFrame({"D":[0, 1615],
                                             "E":[1465,0]},
                                            ["D", "E"])

In [33]:
ampl.option["solver"] = "gurobi"
ampl.solve()

solutionxb = ampl.get_data("xb")
solutionxa = ampl.get_data("xa")
solutionxe = ampl.get_data("xe")
solutionya = ampl.get_data("yb")
solutionyb = ampl.get_data("ya")
solutionye = ampl.get_data("ye")

print(f"Solución:\n\t{solutionxb}")
print(f"Solución:\n\t{solutionxa}")
print(f"Solución:\n\t{solutionxe}")
print(f"Solución:\n\t{solutionyb}")
print(f"Solución:\n\t{solutionya}")
print(f"Solución:\n\t{solutionye}")

Error:
	Error executing "solve" command:
	error processing objective maxganancia:
		invalid subscript costTransAlumi['B','B']

For support/feedback go to https://discuss.ampl.com or e-mail <support@ampl.com>

Solución:
	   index0       index1    |      xb     
    'A'            'B'      |      0      
    'A'            'C'      |      0      
    'A'            'D'      |      0      
    'A'            'E'      |      0      
    'B'            'B'      |      0      
    'B'            'C'      |      0      
    'B'            'D'      |      0      
    'B'            'E'      |      0      
    'C'            'B'      |      0      
    'C'            'C'      |      0      
    'C'            'D'      |      0      
    'C'            'E'      |      0      

Solución:
	   index0       index1       index2    |      xa     
    'B'            'D'             1       |      0      
    'B'            'D'             2       |      0      
    'B'            'D'             3     

## Punto 2

### Asignación de estaciones de bomberos

In [9]:
%%ampl_eval
reset; # limpiar definiciones anteriores
# definir el conjunto de Estaciones
set E;

# definir el conjunto de Sectores
set S;

# definir el tiempo de respuesta de la estacion para el secto e estación s
param tresp{E,S};

# definir frecuencia de incendios del sector s
param fincen{S};

#definir valor muy grande
param M;


# definir las variables de decision:
# x[E,S] asignación de la estacion e para el sector s
var x{E, S} binary;

# La función objetivo es minimizar el tiempo de respuesta
minimize min_tiempo: sum{s in S, e in E} fincen[s] * tresp[s,e] * x[e,s];

# restricciones:

subject to dos_estaciones_activas: sum{e in E} x[e,e]  = 2;

subject to cinco_sectores_asigados: sum{e in E, s in S} x[e,s]=5;

subject to un_sector_no_asignado {e in E}: sum{s in S} x[e,s] <= M*x[e,e];


In [10]:
import pandas as pd

ampl.set["E"] = [i+1 for i in range(5)]
ampl.set["S"] = [i+1 for i in range(5)]
ampl.param["fincen"] = {1:2, 2:1, 3:3, 4:1, 5:3}
ampl.param["M"] = 10e10

tresp_df = pd.DataFrame(
    {
        1: [5, 12, 30, 20, 15],
        2: [20, 4, 15, 10, 25],
        3: [15, 20, 6, 15, 12],
        4: [25, 15, 25, 4, 10],
        5: [10, 25, 15, 12, 5],
    },
    [i+1 for i in range(5)],
)

# To load the data for the parameter happiness you can use either happiness_df or happiness_df_stacked in the same way
ampl.param["tresp"] = tresp_df

display(tresp_df)  # display 2D Pandas DataFrame

,1,2,3,4,5
1,5,20,15,25,10
2,12,4,20,15,25
3,30,15,6,25,15
4,20,10,15,4,12
5,15,25,12,10,5


In [11]:
ampl.option["solver"] = "gurobi"
ampl.solve()

solution = ampl.get_data("{e in E, s in S: x[e, s] > 0} x[e, s]").to_dict()
print(f"Solución:\n\t{solution}")

print("\nPara minimizar los tiempos:")
for e, s in solution:
    print(f"- estación {e} se encarga del sector {s}.")

Gurobi 11.0.3: Gurobi 11.0.3: optimal solution; objective 56
9 simplex iterations
1 branching node
Solución:
	{(1, 1): 1, (1, 2): 1, (1, 4): 1, (2, 2): 1, (2, 4): 1}

Para minimizar los tiempos:
- estación 1 se encarga del sector 1.
- estación 1 se encarga del sector 2.
- estación 1 se encarga del sector 4.
- estación 2 se encarga del sector 2.
- estación 2 se encarga del sector 4.


## Punto 3

### Proceso de creación de petróleo

In [12]:
%%ampl_eval
reset; # limpiar definiciones anteriores
# definir el conjunto de procesos
set PROC;

# definir el conjunto de materias primas
set M;

# definir el conjunto de plantas
set PROD;

# definir la presion de vapor maxima para el producto
param preProd{PROD};

# definir la presion de actanaje minima para el producto
param octProd{PROD};

#definir la demanda minima del producto
param demMin{PROD};

#definir el costo de venta del producto
param venta{PROD};

#definir la demanda maxima del producto
param demMax{PROD};

#definir la presion de vapor de la materia prima
param preMat{M};

#definir octanaje de la materia prima
param octMat{M};

#definir la disponibilidad de la materia prima
param disp{M};

#definir el costo por galon de la materia prima
param costo{M};

#definir el rendimiento de crear el producto
param rend{PROD,PROC};

#costo de producir el producto
param costoProc{PROC};

#parametro para elimiar de una restricción

set prod_without_element within PROD;

# definir las variables de decision:
# xm[M,PROC] cantidad de materia M necesaria para producir para el proceso PROC
var xm{M, PROC} >= 0;

# xp[PROC, PROD] Cantidad de producto PROD hecho en el proceso PROC
var xp{PROC, PROD} >= 0;

# La función objetivo es maximizar la venta
maximize max_venta: sum{prod in PROD, proc in PROC} xp[proc,prod]*venta[prod]- sum{m in M, proc in PROC} xm[m,proc]*(costo[m]+costoProc[proc]);

# restricciones:

subject to demanda_min_producto {prod in PROD}: sum{proc in PROC} xp[proc, prod]  >= demMin[prod];

subject to demanda_max_producto {prod in PROD}: sum{proc in PROC} xp[proc, prod] <= demMax[prod];

subject to disponibilidad_materia {m in M}: sum{proc in PROC} xm[m,proc] <= disp[m];

subject to cumplimiento_presion_vapor {prod in prod_without_element}: (sum {m in M, proc in PROC} xm[m,proc]*rend[prod, proc]*preMat[m]) / (sum {m in M, proc in PROC} xm[m,proc]*rend[prod, proc]) <= preProd[prod];

subject to cumplimiento_octanaje {prod in prod_without_element}: (sum {m in M, proc in PROC} xm[m,proc]*rend[prod, proc]*octMat[m]) / (sum {m in M, proc in PROC} xm[m,proc]*rend[prod, proc])  >= octProd[prod];

subject to ingualdad {prod in PROD, proc in PROC}: (sum {m in M} xm[m,proc]*rend[prod, proc]) =  xp[proc, prod];

In [13]:
import pandas as pd
from math import inf

ampl.set["PROC"] = [i+1 for i in range(3)]
ampl.set["M"] = ["Nacional", "Importada"]
ampl.set["PROD"] = ["Gasolina Regular", "Gasolina Extra", "Otros Productos"]
ampl.param["preProd"] = {"Gasolina Regular":23, "Gasolina Extra":23, "Otros Productos":inf}
ampl.param["octProd"] = {"Gasolina Regular":88, "Gasolina Extra":93, "Otros Productos":0}
ampl.param["demMin"] = {"Gasolina Regular":2100000, "Gasolina Extra":750000, "Otros Productos":1000000}
ampl.param["demMax"] = {"Gasolina Regular":4200000, "Gasolina Extra":1500000, "Otros Productos":2000000}
ampl.param["venta"] = {"Gasolina Regular":2500, "Gasolina Extra":3200, "Otros Productos":1100}
ampl.param["preMat"] = {"Nacional":20 , "Importada":15}
ampl.param["octMat"] = {"Nacional":95 , "Importada":98}
ampl.param["disp"] = {"Nacional":4000000 , "Importada":2000000}
ampl.param["costo"] = {"Nacional":1030 , "Importada":2170}
ampl.param["costoProc"] = {1:250, 2:340, 3:300}

ampl.set["prod_without_element"] = ["Gasolina Regular", "Gasolina Extra"]

rend_df = pd.DataFrame(
    {
        1: [0.60, 0, 0.40],
        2: [0, 0.45, 0.55],
        3: [0.40, 0.25, 0.35],

    },
     ["Gasolina Regular", "Gasolina Extra", "Otros Productos"],
)

# To load the data for the parameter happiness you can use either happiness_df or happiness_df_stacked in the same way
ampl.param["rend"] = rend_df
display(rend_df)  # display 2D Pandas DataFrame

,1,2,3
Gasolina Regular,0.6,0.00,0.40
Gasolina Extra,0.0,0.45,0.25
Otros Productos,0.4,0.55,0.35


In [14]:
ampl.option["solver"] = "gurobi"
ampl.solve()


# solutionxm = ampl.get_data("{proc in PROC, m in M: xm{m, proc} > 0} xm[m, proc]")
# solutionxp = ampl.get_data("{proc in PROC, prod in PROD: xp{proc, prod} > 0} xp[proc, prod]")

solutionxm = ampl.get_data("xm")
solutionxp = ampl.get_data("xp")

print(f"Solución:\n\t{solutionxm}")
print(f"Solución:\n\t{solutionxp}")

print("\nPara maximiza las ventas:")
for m, proc, res in solutionxm:
  print(f"- Cantidad de {m} para producir en el proceso {proc} es: {res}.")

for proc, prod, res in solutionxp:
  print(f"- Cantidad de producto {prod} vendido gracias al proceso {proc} es: {res}.")


Gurobi 11.0.3: Gurobi 11.0.3: optimal solution; objective 3063750000
11 simplex iterations
1 branching node
Solución:
	   index0       index1    |      xm     
'Importada'         1       |      0      
'Importada'         2       |      0      
'Importada'         3       |   1250000   
 'Nacional'         1       |      0      
 'Nacional'         2       |      0      
 'Nacional'         3       |   4000000   

Solución:
	   index0       index1    |      xp     
     1         'Gasolina Extra' |      0      
     1         'Gasolina Regular' |      0      
     1         'Otros Productos' |      0      
     2         'Gasolina Extra' |      0      
     2         'Gasolina Regular' |      0      
     2         'Otros Productos' |      0      
     3         'Gasolina Extra' |   1312500   
     3         'Gasolina Regular' |   2100000   
     3         'Otros Productos' |   1837500   


Para maximiza las ventas:
- Cantidad de Importada para producir en el proceso 1 

## Punto 4

### Distribución de bienes de producción

In [15]:
%%ampl_eval
reset; # limpiar definiciones anteriores
# definir el conjunto de Plantas
set P;

# definir el conjunto de distribuidoras
set D;

# definir la demanda minima de la distribuidora
param dem{D};

# definir la capacidad maxima de producto para la planta
param cap{P};

#definir costo de produccion unitario de producto en la planta
param cos{P};

#definir costo de distribucion unitario de producto en la planta
param dis{P,D};


# definir las variables de decision:
# x[E,S] acantidad de producto a producir desde la planta
var x{P, D} >= 0 integer;

# La función objetivo es minimizar el costo
minimize min_costo: sum{d in D, p in P} x[p,d]*(cos[p]+dis[p,d]);

# restricciones:

subject to no_exeder_capacidad_plan {p in P}: sum{d in D} x[p,d]  <= cap[p];

subject to cumplimiento_distribuidoras {d in D}: sum{p in P} x[p,d] = dem[d];



In [16]:
import pandas as pd

ampl.set["P"] = [i+1 for i in range(3)]
ampl.set["D"] = [i+1 for i in range(5)]
ampl.param["dem"] = {1:2700, 2:2700, 3:9000, 4:4500, 5:3600}
ampl.param["cap"] = {1:4500, 2:9000, 3:11250}
ampl.param["cos"] = {1:60, 2:30, 3:54}

dis_df = pd.DataFrame(
    {
        1: [1.5, 2.4, 3.0],
        2: [2.1, 1.8, 2.7],
        3: [3.3, 3.0, 2.7],
        4: [4.5, 3.6, 3.0],
        5: [4.8, 4.5, 4.8],
    },
    [i+1 for i in range(3)],
)

# To load the data for the parameter happiness you can use either happiness_df or happiness_df_stacked in the same way
ampl.param["dis"] = dis_df

display(dis_df)  # display 2D Pandas DataFrame

,1,2,3,4,5
1,1.5,2.1,3.3,4.5,4.8
2,2.4,1.8,3.0,3.6,4.5
3,3.0,2.7,2.7,3.0,4.8


In [17]:
ampl.option["solver"] = "gurobi"
ampl.solve()

solution = ampl.get_data("{p in P, d in D: x[p, d] > 0} x[p, d]").to_dict()
print(f"Solución:\n\t{solution}")

print("\nPara minimizar los costos:")
for p, d in solution:
    print(f"- Planta {p} envia a la distribuidora {d} la cantidad de {solution[(p,d)]}.")

Gurobi 11.0.3: Gurobi 11.0.3: optimal solution; objective 1076490
7 simplex iterations
1 branching node
Solución:
	{(1, 1): 2250, (2, 1): 450, (2, 2): 2700, (2, 3): 2250, (2, 5): 3600, (3, 3): 6750, (3, 4): 4500}

Para minimizar los costos:
- Planta 1 envia a la distribuidora 1 la cantidad de 2250.
- Planta 2 envia a la distribuidora 1 la cantidad de 450.
- Planta 2 envia a la distribuidora 2 la cantidad de 2700.
- Planta 2 envia a la distribuidora 3 la cantidad de 2250.
- Planta 2 envia a la distribuidora 5 la cantidad de 3600.
- Planta 3 envia a la distribuidora 3 la cantidad de 6750.
- Planta 3 envia a la distribuidora 4 la cantidad de 4500.
